### Como decidir entre remover linhas e imputar pela média
- Se a porcentagem de faltantes (`perc_na`) em uma variável quantitativa for alta, a imputação pela média tende a distorcer a variância e achataria a distribuição. Prefira remoção se a amostra continuar adequada.
- Se for baixa (ex.: < 5%-10%) e você precisa de mais linhas para modelos, imputar média pode ser aceitável, observando a mudança em `variancia`, `desvio`, `q1/q3` e `p5/p95`.
- Diferença: a **variância** mede a dispersão média ao quadrado (unidade ao quadrado); o **desvio padrão** é a raiz da variância (mesma unidade da variável), logo mais interpretável.
- Para variáveis categóricas, avalie se faz sentido criar uma categoria 'Desconhecido' dependendo do uso.
- Use `comparacao_q` para ver como `media`, `mediana`, `variancia`, `desvio` e separatrizes mudam em cada variável.



In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [35]:
from pathlib import Path

pd.set_option('display.float_format', lambda v: f"{v:,.4f}")
BASE_DIR = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
DATA_PATH = BASE_DIR / 'coronarias.xls'  # ajuste se necessário



In [36]:
# Carregar dados e normalizar valores faltantes '.'
# engine='xlrd' pode ser necessário em alguns ambientes; deixamos automático
xls_path = DATA_PATH
assert xls_path.exists(), f"Arquivo não encontrado: {xls_path}"

# Ler a planilha específica 'dados'
df_raw = pd.read_excel(xls_path, sheet_name='dados')

# Normalização robusta de faltantes: '.', strings vazias e variações com espaços
def normalize_missing(val):
    if isinstance(val, str):
        v = val.strip()
        if v == '' or v == '.':
            return np.nan
    return val


In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


try:
    # Carrega o arquivo Excel
    df = pd.read_excel('coronarias.xls', sheet_name='dados')
    print("✅ Arquivo carregado com sucesso!")
    print(f"📊 Dimensões do dataset: {df.shape}")
except Exception as e:
    print(f"❌ Erro ao carregar o arquivo: {e}")
    

✅ Arquivo carregado com sucesso!
📊 Dimensões do dataset: (1500, 70)


In [38]:
# Contar o número de registros preenchidos com o caractere "." em cada coluna
num_pontos_por_coluna = (df == '.').sum()
print("Número de registros preenchidos com '.' por coluna:")
filtro = num_pontos_por_coluna[num_pontos_por_coluna < 500]
print(filtro.sort_values(ascending=False))
print(filtro.sum())

Número de registros preenchidos com '.' por coluna:
C/H        482
C/H-S      482
C/H-M      482
HDLM       478
HDL        478
HDLS       478
GLICM      306
GLICS      306
TRIGM      291
TRIG       291
TRIGS      291
COLM       253
COLS       253
COL        253
GLIC       231
TABAG4     130
TABAG      130
DIAB       111
ALTURA      78
OBESO       78
IMC         78
HA          52
PESO        41
ARTER       19
PDR          7
PSR          7
CAT          0
DA           0
CD           0
IMP          0
ANGINS       0
AH           0
ANGEST       0
IDENT        0
ARRIT        0
ICC          0
IMA          0
IDADE1       0
DI           0
AH3          0
AH2          0
SELO         0
LO3          0
IDA55        0
SEID         0
M_PA         0
M_C          0
CX           0
NUMAL        0
SEXO         0
IDADFEM      0
IDADMAS      0
IDA60        0
SEID6        0
INFARTO      0
dtype: int64
6086


In [39]:
# Seleção das variáveis solicitadas
# Categóricas: SEXO, AH3, INFARTO, ANGEST, DIAB, ARTER, ARRIT, TABAG4, OBESO
# Quantitativas: IDADE1, ALTURA, PESO, IMC, COL, TRIG, PSR, PDR

categoricas = ['SEXO','AH3','INFARTO','ANGEST','DIAB','ARTER','ARRIT','TABAG4','OBESO']
quantitativas = ['IDADE1','ALTURA','PESO','IMC','COL','TRIG','PSR','PDR']

# Filtrar apenas colunas existentes (nome pode variar no arquivo)
cols_existentes = [c for c in categoricas + quantitativas if c in df.columns]
missing_cols = sorted(set(categoricas + quantitativas) - set(cols_existentes))
print('Colunas não encontradas (verificar nomes):', missing_cols)

df_sel = df[cols_existentes].copy()

# Substituir explicitamente '.' e strings vazias por NaN nas colunas selecionadas
df_sel = df_sel.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() in {'.', ''} else x)

# Conversão robusta para numérico (suporta vírgula decimal e espaços)
def to_numeric_locale(series: pd.Series) -> pd.Series:
    if series.dtype.kind in 'biufc':
        return series.astype(float)
    s = series.astype(str).str.strip()
    # tratar '.' isolado já virou NaN acima; aqui focamos em números
    s = s.str.replace(',', '.', regex=False)
    s = s.str.replace(r'[^0-9\.-]', '', regex=True)
    return pd.to_numeric(s, errors='coerce')

for c in quantitativas:
    if c in df_sel.columns:
        df_sel[c] = to_numeric_locale(df_sel[c])

Colunas não encontradas (verificar nomes): []


In [40]:
#codigo que seleciona 500 linhas diferentes do df original e salva em um csv
import random
import math

def criaCsv():
    linhas_selecionadas = set()
    linhas_selecionadas.add(1600)
    nLinhas = 0
    while nLinhas <= 499:
        linha_escolhida = 1600
        while linha_escolhida in linhas_selecionadas:
            linha_escolhida = math.floor(random.random()*len(df_sel))
        linha_raw = df_sel.loc[linha_escolhida]
        nLinhas += 1
        linhas_selecionadas.add(linha_escolhida)
    linhas_selecionadas.remove(1600)
    df_csv = df_sel.loc[list(linhas_selecionadas)]
    df_csv.to_csv('coronarias.csv',sep=';',na_rep='NaN',index=True)

criaCsv()
#O arquivo foi criado e se encontra na pasta.


In [41]:
# abre o csv e conta quantos NaN tem em cada coluna


import random
import math
import numpy as np
import pandas as pd


df_csv_final = pd.read_csv('coronarias.csv',sep=';',na_values='NaN')
nan_por_coluna = df_csv_final.isna().sum()

#cria csvs aleatorios até a coluna trig ter poucos dados faltantes
while nan_por_coluna['TRIG'] >= 80:
    criaCsv()
    df_csv_final = pd.read_csv('coronarias.csv',sep=';',na_values='NaN')
    nan_por_coluna = df_csv_final.isna().sum()

print(nan_por_coluna)


Unnamed: 0     0
SEXO           0
AH3            0
INFARTO        0
ANGEST         0
DIAB          34
ARTER          7
ARRIT          0
TABAG4        42
OBESO         17
IDADE1         0
ALTURA        17
PESO           5
IMC           17
COL           60
TRIG          76
PSR            3
PDR            3
dtype: int64
